In [2]:
%load_ext sql
%sql duckdb:///data/vermont.duckdb
%config SqlMagic.named_parameters="enabled"

Connecting to 'duckdb:///data/vermont.duckdb'

In [2]:
%%sql

-- read raw data into duckdb
DROP TABLE IF EXISTS sightings_raw;

CREATE TABLE sightings_raw AS
SELECT * 
FROM read_csv_auto('data/ebd_US-VT_smp_relJul-2025.txt',
    delim='\t', sample_size=-1);

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [3]:
%%sql

SELECT COUNT(*) AS total_records FROM sightings_raw;

Running query in 'duckdb:///data/vermont.duckdb'

total_records
10580275


In [15]:
%%sql

SELECT * FROM sightings_raw WHERE CATEGORY == 'form' AND "COMMON NAME" != 'Redpoll';

Running query in 'duckdb:///data/vermont.duckdb'

GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER,CATEGORY,TAXON CONCEPT ID,COMMON NAME,SCIENTIFIC NAME,SUBSPECIES COMMON NAME,SUBSPECIES SCIENTIFIC NAME,EXOTIC CODE,OBSERVATION COUNT,BREEDING CODE,BREEDING CATEGORY,BEHAVIOR CODE,AGE/SEX,COUNTRY,COUNTRY CODE,STATE,STATE CODE,COUNTY,COUNTY CODE,IBA CODE,BCR CODE,USFWS CODE,ATLAS BLOCK,LOCALITY,LOCALITY ID,LOCALITY TYPE,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,OBSERVER ORCID ID,SAMPLING EVENT IDENTIFIER,OBSERVATION TYPE,PROTOCOL NAME,PROTOCOL CODE,PROJECT NAMES,PROJECT IDENTIFIERS,DURATION MINUTES,EFFORT DISTANCE KM,EFFORT AREA HA,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,HAS MEDIA,APPROVED,REVIEWED,REASON,CHECKLIST COMMENTS,SPECIES COMMENTS,column52
URN:CornellLabOfOrnithology:EBIRD:OBS458926892,2021-04-01 19:23:09.561087,6541,form,avibase-12D5BA6C,Iceland Gull,Larus glaucoides,Iceland Gull (kumlieni/glaucoides),Larus glaucoides kumlieni/glaucoides,None,1,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Burlington CBC,L5281730,P,44.4768291,-73.2217312,1977-12-18,None,obsr50466,None,S33808834,Historical,Historical,P62,None,None,None,None,None,None,1,None,0,1,1,None,None,Details lost,None
URN:CornellLabOfOrnithology:EBIRD:OBS245810947,2021-02-24 11:34:39.783867,6541,form,avibase-12D5BA6C,Iceland Gull,Larus glaucoides,Iceland Gull (kumlieni/glaucoides),Larus glaucoides kumlieni/glaucoides,None,X,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Burlington Intervale,L246386,H,44.4964925,-73.2062496,1978-04-01,None,obsr27544,None,S17903431,Incidental,Incidental,P20,None,None,None,None,None,None,0,None,0,1,1,None,Walter Ellison,None,None
URN:CornellLabOfOrnithology:EBIRD:OBS245809569,2021-02-24 10:37:29.792970,6541,form,avibase-12D5BA6C,Iceland Gull,Larus glaucoides,Iceland Gull (kumlieni/glaucoides),Larus glaucoides kumlieni/glaucoides,None,1,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Burlington,L131933,T,44.4758492,-73.2120667,1978-04-01,None,obsr27544,None,S17903302,Incidental,Incidental,P20,None,None,None,None,None,None,0,None,0,1,1,None,Walter Ellison,None,None
URN:CornellLabOfOrnithology:EBIRD:OBS245793167,2024-09-09 11:42:05.289468,6541,form,avibase-12D5BA6C,Iceland Gull,Larus glaucoides,Iceland Gull (kumlieni/glaucoides),Larus glaucoides kumlieni/glaucoides,None,1,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Burlington,L131933,T,44.4758492,-73.2120667,1979-04-04,None,obsr27544,None,S17901230,Incidental,Incidental,P20,None,None,None,None,None,2,0,None,0,1,1,None,Wayne Scott,None,None
URN:CornellLabOfOrnithology:EBIRD:OBS137866526,2021-02-24 10:58:27.800229,6541,form,avibase-12D5BA6C,Iceland Gull,Larus glaucoides,Iceland Gull (kumlieni/glaucoides),Larus glaucoides kumlieni/glaucoides,None,X,None,None,None,None,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,Charlotte,L165639,T,44.30975,-73.26097,1980-01-03,None,obsr27544,None,S9721139,Incidental,Incidental,P20,None,None,None,None,None,1,0,None,0,1,1,None,None,None,None
URN:CornellLabOfOrnithology:EBIRD:OBS42838738,2017-01-07 20:17:00,5446,form,avibase-9BEECB7E,American Coot,Fulica americana,American Coot (Red-shielded),Fulica americana (Red-shielded),None,X,None,None,None,None,United States,US,Vermont,US-VT,Orleans,US-VT-019,None,14,None,None,Lake Memphremagog - Derby (5966 acres),L752020,H,44.9731378,-72.215948,1980-05-28,None,obsr27544,None,S3051025,Incidental,Incidental,P20,None,None,None,None,None,None,0,None,0,1,1,None,RSNR- accepted as possible- 4/83G. Frank Oatmanthree,None,None
URN:CornellLabOfOrnithology:EBIRD:OBS42839008,2015-02-24 11:43:13,5446,form,avibase-9BEECB7E,American Coot,Fulica americana,American Coot (Red-shielded),Fulica americana (Red-shielded),None,X,None,None,None,None,United States,US,Vermont,US-VT,Rutland,US-VT-021,US-VT_7,14,None,None,West Rutland Marsh IB

In [17]:
%%sql

DROP TABLE IF EXISTS sightings_staging;
CREATE TABLE sightings_staging AS
SELECT
    TRY_CAST(REGEXP_EXTRACT("GLOBAL UNIQUE IDENTIFIER", '(\d+)$') AS BIGINT) AS global_id,
    TRY_CAST("LAST EDITED DATE" AS DATE) AS last_edited_date,
    "TAXONOMIC ORDER" as taxonomic_order,
    "CATEGORY" as species_category,
    "COMMON NAME" as common_name,
    "SCIENTIFIC NAME" as scientific_name,
    TRY_CAST("OBSERVATION COUNT" AS INT) AS observation_count,
    "COUNTRY" as country,
    "COUNTRY CODE" as country_code,
    "STATE" as state,
    "STATE CODE" as state_code,
    "COUNTY" as county,
    "COUNTY CODE" as county_code,
    "LOCALITY" as locality,
    TRY_CAST(REGEXP_EXTRACT("LOCALITY ID", '(\d+)$') AS BIGINT) AS locality_id,
    "LOCALITY TYPE" as locality_type,
    "LATITUDE"::FLOAT AS latitude,
    "LONGITUDE"::FLOAT AS longitude,
    TRY_CAST("OBSERVATION DATE" AS DATE) AS observation_date,
    TRY_CAST("TIME OBSERVATIONS STARTED" AS TIME) AS time_observations_started,
    TRY_CAST(REGEXP_EXTRACT("OBSERVER ID", '(\d+)$') AS BIGINT) AS observer_id,
    TRY_CAST(REGEXP_EXTRACT("SAMPLING EVENT IDENTIFIER", '(\d+)$') AS BIGINT) AS sampling_id,
    "OBSERVATION TYPE" as observation_type,
    "DURATION MINUTES"::INT AS duration_minutes,
    "EFFORT DISTANCE KM"::FLOAT AS effort_distance_km,
    "NUMBER OBSERVERS"::INT AS number_observers,
    "ALL SPECIES REPORTED"::BOOLEAN as all_species_reported,
    TRY_CAST(REGEXP_EXTRACT("GROUP IDENTIFIER", '(\d+)$') AS BIGINT) AS group_id  
FROM sightings_raw
WHERE locality_type == 'H' AND
    (species_category == 'species' OR species_category == 'issf' OR species_category == 'form') AND
    all_species_reported IS TRUE
;


Running query in 'duckdb:///data/vermont.duckdb'

Count


In [40]:
%%sql

SELECT COUNT(*) as num_rows
FROM sightings_staging;

Running query in 'duckdb:///data/vermont.duckdb'

num_rows
4341571


In [41]:
%%sql

-- drop duplicate observations from group sightings
DROP TABLE IF EXISTS sightings_clean;
CREATE TABLE sightings_clean AS
SELECT *
FROM (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY group_id, common_name
            ORDER BY sampling_id --figure out something else to order by?
        ) as row_num
    FROM sightings_staging
) t
WHERE group_id IS NULL or row_num = 1;

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [42]:
%%sql

SELECT COUNT(*) AS total_records
FROM sightings_clean;

Running query in 'duckdb:///data/vermont.duckdb'

total_records
3407220


In [43]:
%%sql
SELECT COUNT(*) as num_checklists
FROM (
    SELECT COUNT(*) FROM sightings_clean
    GROUP BY sampling_id
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_checklists
249494


In [10]:
%%sql

DROP TABLE IF EXISTS checklists_raw;
CREATE TABLE checklists_raw AS
SELECT *
FROM read_csv_auto('data/ebd_US-VT_smp_relJul-2025_sampling.txt', delim='\t')

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [11]:
%%sql

SELECT *
FROM checklists_raw
LIMIT 5;

Running query in 'duckdb:///data/vermont.duckdb'

LAST EDITED DATE,COUNTRY,COUNTRY CODE,STATE,STATE CODE,COUNTY,COUNTY CODE,IBA CODE,BCR CODE,USFWS CODE,ATLAS BLOCK,LOCALITY,LOCALITY ID,LOCALITY TYPE,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,OBSERVER ID,OBSERVER ORCID ID,SAMPLING EVENT IDENTIFIER,OBSERVATION TYPE,PROTOCOL NAME,PROTOCOL CODE,PROJECT NAMES,PROJECT IDENTIFIERS,DURATION MINUTES,EFFORT DISTANCE KM,EFFORT AREA HA,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,CHECKLIST COMMENTS,column33
2025-03-28 14:58:23.718586,United States,US,Vermont,US-VT,Chittenden,US-VT-007,None,13,None,None,New Village Farm,L11398723,P,44.3860168,-73.2422018,2025-03-28,13:45:00,obsr901147,None,S221347013,Traveling,Traveling,P22,None,None,30,0.483,None,2,1,None,None,None
2025-07-02 16:01:44.022663,United States,US,Vermont,US-VT,Windham,US-VT-025,None,14,None,None,Hogback Mountain Overlook,L11360827,P,42.8527281,-72.7934659,2025-07-02,16:01:00,obsr604565,None,S256290564,Incidental,Incidental,P20,None,None,None,None,None,1,0,None,None,None
2025-04-09 15:49:20.512607,United States,US,Vermont,US-VT,Windham,US-VT-025,None,14,None,None,Hogback Mountain Overlook,L11360827,P,42.8527281,-72.7934659,2025-04-09,15:49:00,obsr604565,None,S224162872,Incidental,Incidental,P20,None,None,None,None,None,1,0,None,None,None
2025-05-26 10:18:02.301179,United States,US,Vermont,US-VT,Franklin,US-VT-011,None,14,None,None,Fairfield Swamp WMA,L1140162,H,44.7955561,-72.9962755,2025-05-26,10:06:00,obsr2046614,None,S242975716,Traveling,Traveling,P22,None,None,10,0.259,None,1,1,None,None,None
2025-06-13 07:31:20.354768,United States,US,Vermont,US-VT,Franklin,US-VT-011,None,14,None,None,Fairfield Swamp WMA,L1140162,H,44.7955561,-72.9962755,2025-06-13,07:23:00,obsr459569,None,S249781342,Stationary,Stationary,P21,None,None,7,None,None,1,1,None,None,None


In [12]:
%%sql

DROP TABLE IF EXISTS checklists_staging;
CREATE TABLE checklists_staging AS
SELECT
    TRY_CAST("LAST EDITED DATE" AS DATE) AS last_edited_date,
    "COUNTRY" as country,
    "COUNTRY CODE" as country_code,
    "STATE" as state,
    "STATE CODE" as state_code,
    "COUNTY" as county,
    "COUNTY CODE" as county_code,
    "LOCALITY" as locality,
    TRY_CAST(REGEXP_EXTRACT("LOCALITY ID", '(\d+)$') AS BIGINT) AS locality_id,
    "LOCALITY TYPE" as locality_type,
    "LATITUDE"::FLOAT AS latitude,
    "LONGITUDE"::FLOAT AS longitude,
    TRY_CAST("OBSERVATION DATE" AS DATE) AS observation_date,
    TRY_CAST("TIME OBSERVATIONS STARTED" AS TIME) AS time_observations_started,
    TRY_CAST(REGEXP_EXTRACT("OBSERVER ID", '(\d+)$') AS BIGINT) AS observer_id,
    TRY_CAST(REGEXP_EXTRACT("SAMPLING EVENT IDENTIFIER", '(\d+)$') AS BIGINT) AS sampling_id,
    "OBSERVATION TYPE" as observation_type,
    "DURATION MINUTES"::INT AS duration_minutes,
    "EFFORT DISTANCE KM"::FLOAT AS effort_distance_km,
    "NUMBER OBSERVERS"::INT AS number_observers,
    "ALL SPECIES REPORTED"::BOOLEAN as all_species_reported,
    TRY_CAST(REGEXP_EXTRACT("GROUP IDENTIFIER", '(\d+)$') AS BIGINT) AS group_id
FROM checklists_raw
WHERE
    locality_type == 'H' AND
    all_species_reported IS TRUE;

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [13]:
%%sql

SELECT COUNT(*) AS total_checklists
FROM checklists_staging;

Running query in 'duckdb:///data/vermont.duckdb'

total_checklists
306132


In [14]:
%%sql

-- drop duplicate checklists from groups
DROP TABLE IF EXISTS checklists_clean;
CREATE TABLE checklists_clean AS
SELECT *
FROM (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY group_id
            ORDER BY sampling_id --figure out something else to order by?
        ) as row_num
    FROM checklists_staging
) t
WHERE group_id IS NULL or row_num = 1

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [16]:
%%sql

DROP TABLE IF EXISTS locality_lookup;
CREATE TABLE locality_lookup AS
WITH checklist_counts AS (
    SELECT locality_id, COUNT(*) AS total_checklists
    FROM checklists_clean
    GROUP BY locality_id
)
SELECT DISTINCT c.locality, c.locality_id, c.locality_type, c.latitude, c.longitude, cc.total_checklists
FROM checklists_clean c
JOIN checklist_counts cc
    ON c.locality_id = cc.locality_id
ORDER BY cc.total_checklists DESC;

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [15]:
%%sql

SELECT COUNT(*) AS unique_checklists
FROM checklists_clean;

Running query in 'duckdb:///data/vermont.duckdb'

unique_checklists
249179


In [ ]:
%%sql
-- most of these should be fixed by ordering by a different field when deduplicating
SELECT COUNT(*) AS num_checklists_missing
FROM (
    SELECT sampling_id
    FROM checklists_clean
    EXCEPT
    SELECT DISTINCT sampling_id
    FROM sightings_clean
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_checklists_missing
1170


In [ ]:
%%sql
-- most of these should be fixed by ordering by a different field when deduplicating
SELECT COUNT(*) AS num_checklists_missing
FROM (
    SELECT sampling_id
    FROM sightings_clean
    EXCEPT
    SELECT DISTINCT sampling_id
    FROM checklists_clean
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_checklists_missing
1485


In [12]:
%%sql
DROP TABLE IF EXISTS detection_frequencies;
CREATE TABLE detection_frequencies AS
WITH checklists AS (
    SELECT
        locality,
        locality_id,
        DAYOFYEAR(observation_date) AS day_of_year,
        COUNT(DISTINCT sampling_id) AS total_checklists
    FROM sightings_clean
    GROUP BY locality, locality_id, day_of_year
), detections AS (
    SELECT
        locality_id,
        DAYOFYEAR(observation_date) AS day_of_year,
        common_name,
        COUNT(DISTINCT sampling_id) AS total_detections
    FROM sightings_clean
    GROUP BY locality_id, day_of_year, common_name
), species AS (
    SELECT DISTINCT
        locality_id,
        common_name
    FROM sightings_clean
)

SELECT
    c.locality,
    c.locality_id,
    c.day_of_year,
    s.common_name,
    COALESCE(d.total_detections, 0) AS total_detections,
    c.total_checklists
FROM checklists c
JOIN species s
    ON s.locality_id = c.locality_id
LEFT JOIN detections d
    ON d.locality_id = c.locality_id AND d.day_of_year = c.day_of_year AND d.common_name = s.common_name
ORDER BY c.locality_id, c.day_of_year DESC;

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [62]:
%%sql
DROP TABLE IF EXISTS rolling_avg_freq;
CREATE TABLE rolling_avg_freq AS
WITH wrapped AS (
    SELECT
        *,
        day_of_year AS wrapped_day_of_year
    FROM detection_frequencies

    UNION ALL

    SELECT
        *,
        day_of_year + 366 AS wrapped_day_of_year
    FROM detection_frequencies
    WHERE day_of_year <= 6
)

SELECT
    locality,
    locality_id,
    day_of_year,
    common_name,
    total_detections,
    total_checklists,
    SUM(total_detections) OVER w / SUM(total_checklists) OVER w as rolling_avg_freq
FROM wrapped
WHERE 4 <= wrapped_day_of_year AND wrapped_day_of_year <= 369
WINDOW w AS (
    PARTITION BY locality_id, common_name
    ORDER BY wrapped_day_of_year
    RANGE BETWEEN 3 PRECEDING AND 3 FOLLOWING
)
ORDER BY day_of_year
;

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [63]:
%%sql
SELECT *
FROM rolling_avg_freq
WHERE locality = 'Delta Park IBA' AND common_name = 'Black-capped Chickadee'
;

Running query in 'duckdb:///data/vermont.duckdb'

locality,locality_id,day_of_year,common_name,total_detections,total_checklists,rolling_avg_freq
Delta Park IBA,165376,1,Black-capped Chickadee,3,4,0.7058823529411765
Delta Park IBA,165376,2,Black-capped Chickadee,2,3,0.75
Delta Park IBA,165376,3,Black-capped Chickadee,1,2,0.6666666666666666
Delta Park IBA,165376,4,Black-capped Chickadee,2,3,0.6666666666666666
Delta Park IBA,165376,5,Black-capped Chickadee,1,3,0.7222222222222222
Delta Park IBA,165376,6,Black-capped Chickadee,3,5,0.782608695652174
Delta Park IBA,165376,7,Black-capped Chickadee,4,4,0.8148148148148148
Delta Park IBA,165376,8,Black-capped Chickadee,3,3,0.8571428571428571
Delta Park IBA,165376,9,Black-capped Chickadee,5,5,0.92
Delta Park IBA,165376,10,Black-capped Chickadee,4,4,0.92
